<a href="https://colab.research.google.com/github/candidobri2/datasetonibus/blob/main/An%C3%A1lise_onibus_fortaleza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de demanda dos terminais e linhas de ônibus da cidade de Fortaleza - José Candido Brito Neto

## E agora o que eu vou fazer?

- Qual é a proporção de pagantes x gratuidades e meias-entradas nos ônibus de Fortaleza? FEITO
- Quais são as linhas que possuem a maior e menor demanda de Fortaleza? (Nos dias úteis e nos fins de semana) (levando em consideração que alguns ônibus tem mais viagens que outros) FEITO
- Relação dos ônibus que mais recebem estudantes e dos ônibus que mais recebem idosos percentualmente.
- Comparativo entre os terminais rodoviários de passageiros.
- Como funciona a sazionalidade durante a semana, quais dias são mais lotados digamos assim.
- Quais são os ônibus com maior média de passageiros por viagem (vou precisar saber quantas viagens cada ônibus fez)
- Verificar se encontro algum outlier em determinados dias de alguma ou algumas linhas.


Esse é o meu primeiro projeto de análise de dados usando Python. Nele, eu usarei os dados disponibilizados pela ETUFOR, que constam o número de passageiros que passaram pelos terminais ou linhas de ônibus durante os meses de janeiro e fevereiro de 2022.

In [ ]:
import pandas as pd
import numpy as np

Primeiro vamos carregar os dados das duas planilhas no formato CSV (comma separated values)

# Limpeza dos dados

## Importando a demanda de janeiro, ajeitando as colunas e adicionando um cabeçalho.

In [ ]:
demanda_jan = pd.read_csv("./demanda_202201.csv")

In [ ]:
demanda_jan

,2022-01-01;1;Terminal Antônio Bezerra ;3;Estudante;3
0,2022-01-01;1;Terminal Antônio Bezerra ...
1,2022-01-01;1;Terminal Antônio Bezerra ...
2,2022-01-01;1;Terminal Antônio Bezerra ...
3,2022-01-01;1;Terminal Antônio Bezerra ...
4,2022-01-01;1;Terminal Antônio Bezerra ...
...,...
86028,2022-01-31;920;Papicu/Caça e Pesca ...
86029,2022-01-31;920;Papicu/Caça e Pesca ...
86030,2022-01-31;999;Quilometragem/Terminal ...
86031,2022-01-31;999;Quilometragem/Terminal ...


Quanto importei pela primeira vez, os dados se mostraram sem cabeçalho e com um erro grave, que é não separar as colunas. Tive que ver a questão das colunas e descobri um parametro da read_csv que é o _sep_. O problema acontece porque a separação default do csv é a virgula, e a separação do meu arquivo está como ponto e vírgula. Então vamos mudar.

In [ ]:
demanda_jan = pd.read_csv("./demanda_202201.csv",sep=";")

In [ ]:
demanda_jan

,2022-01-01,1,Terminal Antônio Bezerra,3,Estudante,3.1
0,2022-01-01,1,Terminal Antônio Bezerra,4,Gratuidade Idoso,666
1,2022-01-01,1,Terminal Antônio Bezerra,5,Vale-transporte (vermelho),2
2,2022-01-01,1,Terminal Antônio Bezerra,11,Idoso Identificação,97
3,2022-01-01,1,Terminal Antônio Bezerra,12,Estudante com débito,873
4,2022-01-01,1,Terminal Antônio Bezerra,16,Grat. Deficiente - S/Acomp,4
...,...,...,...,...,...,...
86028,2022-01-31,920,Papicu/Caça e Pesca,32,Gratuidade Infantil,2
86029,2022-01-31,920,Papicu/Caça e Pesca,37,VT Beneficio Piloto BUM,3
86030,2022-01-31,999,Quilometragem/Terminal,4,Gratuidade Idoso,23
86031,2022-01-31,999,Quilometragem/Terminal,11,Idoso Identificação,1


Agora precisamos consertar o problema com o cabeçalho, vou testar adicionando in-place com:

In [ ]:
#demanda_jan[0] = ["Data", "ID", "Nome", "ID Passageiro","Tipo Passageiro","Quantidade" ] ##deu ruim

Olhando no google, vi que podemos adicionar um cabeçalho por meio do método _columns_

In [ ]:
demanda_jan.columns = ["Data", "ID", "Nome", "ID Passageiro","Tipo Passageiro","Quantidade" ] 

In [ ]:
demanda_jan

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade
0,2022-01-01,1,Terminal Antônio Bezerra,4,Gratuidade Idoso,666
1,2022-01-01,1,Terminal Antônio Bezerra,5,Vale-transporte (vermelho),2
2,2022-01-01,1,Terminal Antônio Bezerra,11,Idoso Identificação,97
3,2022-01-01,1,Terminal Antônio Bezerra,12,Estudante com débito,873
4,2022-01-01,1,Terminal Antônio Bezerra,16,Grat. Deficiente - S/Acomp,4
...,...,...,...,...,...,...
86028,2022-01-31,920,Papicu/Caça e Pesca,32,Gratuidade Infantil,2
86029,2022-01-31,920,Papicu/Caça e Pesca,37,VT Beneficio Piloto BUM,3
86030,2022-01-31,999,Quilometragem/Terminal,4,Gratuidade Idoso,23
86031,2022-01-31,999,Quilometragem/Terminal,11,Idoso Identificação,1


## Características do dataframe - demanda_jan | Agora vamos dar uma olhada nas características do dataframe da demanda do mês de janeiro.

In [ ]:
demanda_jan.shape ### Temos 86033 linhas 

(86033, 6)

O dataframe possui mais de 86 mil linhas referentes a informações de diferentes linhas de ônibus e terminais

In [ ]:
demanda_jan.isnull().sum()

Data               0
ID                 0
Nome               0
ID Passageiro      0
Tipo Passageiro    0
Quantidade         0
dtype: int64

In [ ]:
id = demanda_jan["ID"]

In [ ]:
demanda_jan.iloc[86030]

Data                                             2022-01-31
ID                                                      999
Nome               Quilometragem/Terminal                  
ID Passageiro                                             4
Tipo Passageiro                            Gratuidade Idoso
Quantidade                                               23
Name: 86030, dtype: object

In [ ]:
demanda_jan.describe()

,ID,ID Passageiro,Quantidade
count,86033.000000,86033.000000,86033.000000
mean,385.378680,13.905467,145.243639
std,267.756535,10.536421,391.016026
min,1.000000,2.000000,1.000000
25%,102.000000,5.000000,3.000000
50%,359.000000,11.000000,13.000000
75%,632.000000,16.000000,84.000000
max,999.000000,37.000000,6031.000000


Podemos tirar algumas informações daqui. Temos 86.033 quantidades diferentes, o número máximo em um tipo de passageiro foi 6031. Em que ônibus será que foi?


## Importando o dataframe do mês de fevereiro

In [ ]:
dem_fev = pd.read_csv("./demanda_202202.csv",sep=";")

In [ ]:
dem_fev

,DataMovimento,CodLinha,NomeLinha,TipoPassageiro,NomePassageiro,Demanda
0,01/02/2022,1,Terminal Antônio Bezerra,3,Estudante,35
1,01/02/2022,1,Terminal Antônio Bezerra,4,Gratuidade Idoso,2884
2,01/02/2022,1,Terminal Antônio Bezerra,5,Vale-transporte (vermelho),8
3,01/02/2022,1,Terminal Antônio Bezerra,11,Idoso Identificação,411
4,01/02/2022,1,Terminal Antônio Bezerra,12,Estudante com débito,1194
...,...,...,...,...,...,...
80543,28/02/2022,920,Papicu/Caça e Pesca,32,Gratuidade Infantil,1
80544,28/02/2022,920,Papicu/Caça e Pesca,37,VT Beneficio Piloto BUM,2
80545,28/02/2022,999,Quilometragem/Terminal,4,Gratuidade Idoso,2
80546,28/02/2022,999,Quilometragem/Terminal,11,Idoso Identificação,15


Bom, temos algumas surpresas aqui com o mês de fevereiro. Aparentemente temos menos linhas de ônibus ou novos tipos de passageiros, pois o dataframe possui 80548 linhas. E dessa vez os dados possuem cabeçalho, para ficar igual ao anterior eu irei modificar o cabeçalho com o comando columns

In [ ]:
dem_fev.columns = ["Data", "ID", "Nome", "ID Passageiro","Tipo Passageiro","Quantidade" ]

O formato das datas estão diferentes, lá na frente isso me deu problema, então vou ajeitar aqui por meio do _apply_

In [ ]:
dem_fev["Data"] = dem_fev["Data"].apply(lambda x: str(x.split("/")[2])+"-"+str(x.split("/")[1])+"-"+str(x.split("/")[0]))

In [ ]:
dem_fev

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade
0,2022-02-01,1,Terminal Antônio Bezerra,3,Estudante,35
1,2022-02-01,1,Terminal Antônio Bezerra,4,Gratuidade Idoso,2884
2,2022-02-01,1,Terminal Antônio Bezerra,5,Vale-transporte (vermelho),8
3,2022-02-01,1,Terminal Antônio Bezerra,11,Idoso Identificação,411
4,2022-02-01,1,Terminal Antônio Bezerra,12,Estudante com débito,1194
...,...,...,...,...,...,...
80543,2022-02-28,920,Papicu/Caça e Pesca,32,Gratuidade Infantil,1
80544,2022-02-28,920,Papicu/Caça e Pesca,37,VT Beneficio Piloto BUM,2
80545,2022-02-28,999,Quilometragem/Terminal,4,Gratuidade Idoso,2
80546,2022-02-28,999,Quilometragem/Terminal,11,Idoso Identificação,15


In [ ]:
dem_fev.isna().sum() ##Não temos dados vazios.

Data               0
ID                 0
Nome               0
ID Passageiro      0
Tipo Passageiro    0
Quantidade         0
dtype: int64

## PROBLEMA 1: temos mais linhas nesse, ou temos mais tipos de passageiros?

#### Abordagem 1 - Podemos ver a quantidade de valores único na coluna ID

In [ ]:
unico_jan = demanda_jan["ID"].unique()
print(len(unico_jan))

333


Nós temos 333 valores diferentes na coluna ID do dataframe demanda_jan, isso nos sugere (mas não garante) que temos 333 linhas de ônibus diferentes. Agora vamos ver no fevereiro

In [ ]:
dem_fev["ID"].unique().shape

(332,)

Em fevereiro nós temos 332 valores únicos de ID, 1 a menos que janeiro, o que é uma surpresa, vamos ver a intereseção desses dois arrays. Podemos acessar isso por meio do método np.instersect1d

In [ ]:
unico_janeiro = demanda_jan["ID"].unique() #array com valores unicos de ID
unico_fevereiro = dem_fev["ID"].unique() ##array com valores únicos de ID


In [ ]:
np.intersect1d(unico_janeiro,unico_fevereiro).shape #ID que estão nas duas dataframes

(324,)

In [ ]:
diferenca = np.setxor1d(unico_fevereiro,unico_janeiro) 
print(diferenca)


[ 13  62  98 151 221 324 354 363 762 767 770 870 919 946 949 970 986]


In [ ]:
just_in_jan = np.intersect1d(diferenca,unico_janeiro)
print(just_in_jan)

[ 13  62  98 151 324 354 762 767 919]


In [ ]:
justinfev = np.intersect1d(diferenca,unico_fevereiro)
print(justinfev)

[221 363 770 870 946 949 970 986]


Portanto, podemos tirar de conclusão que as linhas 13,62,98,151,324,254,762,767,919 estão na tabela de demanda de janeiro mas não em fevereiro. E as linhas 221,363,770,870,946,949,970,986 estão em fevereiro mas não estão em janeiro. Note que fevereiro tem uma a menos, por isso a diferença que vimos anteriormente.

**Vamos tentar exibir os nome destas linhas de alguma forma**

In [ ]:
demanda_jan.loc[demanda_jan["ID"] == just_in_jan[7]] 

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade
71918,2022-01-26,767,Vila Velha/North Shopping/Riomar Kennedy,2,Popular,2
71919,2022-01-26,767,Vila Velha/North Shopping/Riomar Kennedy,3,Estudante,13
71920,2022-01-26,767,Vila Velha/North Shopping/Riomar Kennedy,4,Gratuidade Idoso,111
71921,2022-01-26,767,Vila Velha/North Shopping/Riomar Kennedy,11,Idoso Identificação,45
71922,2022-01-26,767,Vila Velha/North Shopping/Riomar Kennedy,12,Estudante com débito,103
71923,2022-01-26,767,Vila Velha/North Shopping/Riomar Kennedy,15,Projovem,3
71924,2022-01-26,767,Vila Velha/North Shopping/Riomar Kennedy,16,Grat. Deficiente - S/Acomp,3
71925,2022-01-26,767,Vila Velha/North Shopping/Riomar Kennedy,37,VT Beneficio Piloto BUM,1


In [ ]:
dem_fev.loc[dem_fev["ID"] == justinfev[7]] 

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade
14704,2022-02-05,986,Antônio Bezerra/Halleluya/Castelão,28,Transbordo,22


## Resumo da ópera: acredito que posso tirar essas linhas da contagem porque pelo visto elas não contam tanto assim e são linhas e dias muitos específicos. Outro ponto importante é que as linhas a mais não são justificadas por diferentes nomes de ônibus mas sim por outro coisa que precisamos investigar. Portanto vamos retirar essas linhas do nosso dataframes.

#### Retirar as linhas "ruins" dos dataframes

In [ ]:
for i in just_in_jan:
  demanda_jan.drop(demanda_jan[demanda_jan["ID"]==i].index, inplace=True)
  # pega o index dos elementos de demanda_jan onde demanda_jan["id"] é igual a cada um dos números de just in jan

In [ ]:
demanda_jan["ID"].unique().shape

(324,)

In [ ]:
for i in justinfev:
  dem_fev.drop(dem_fev[dem_fev["ID"]==i].index,inplace=True)

In [ ]:
dem_fev["ID"].unique().shape

(324,)

O código acima "garante" que os dois dataframes possuem a mesma quantidade de IDs únicos. Mas o dem_fec tem mais linhas, vamos saber os valores únicos dos tipos de passageiros e comparar os dois.


In [ ]:
tipos_arr_jan = demanda_jan["ID Passageiro"].unique()

In [ ]:
tipos_arr_fev = dem_fev["ID Passageiro"].unique()

In [ ]:
np.setxor1d(tipos_arr_jan,tipos_arr_fev) #np.setxor1d mostra valores que não pertencem aos dois arrays

array([], dtype=int64)

Isso nos diz que os tipos de passageiros nos meses de janeiro e fevereiro são iguais.

In [ ]:
tipos_arr_jan

array([ 4,  5, 11, 12, 16, 37,  2, 27,  3,  6,  9, 15, 31, 32, 28, 33,  7])

In [ ]:
print(dem_fev.shape[0])
print(demanda_jan.shape[0])

80480
85982


## Uma explicação para fevereiro ter menos linhas é que o mês possui menos dias.

# Juntando os dois dataframes em um só - merge e concat

In [ ]:
demanda = pd.concat([demanda_jan,dem_fev])

In [ ]:
demanda

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade
0,2022-01-01,1,Terminal Antônio Bezerra,4,Gratuidade Idoso,666
1,2022-01-01,1,Terminal Antônio Bezerra,5,Vale-transporte (vermelho),2
2,2022-01-01,1,Terminal Antônio Bezerra,11,Idoso Identificação,97
3,2022-01-01,1,Terminal Antônio Bezerra,12,Estudante com débito,873
4,2022-01-01,1,Terminal Antônio Bezerra,16,Grat. Deficiente - S/Acomp,4
...,...,...,...,...,...,...
80543,2022-02-28,920,Papicu/Caça e Pesca,32,Gratuidade Infantil,1
80544,2022-02-28,920,Papicu/Caça e Pesca,37,VT Beneficio Piloto BUM,2
80545,2022-02-28,999,Quilometragem/Terminal,4,Gratuidade Idoso,2
80546,2022-02-28,999,Quilometragem/Terminal,11,Idoso Identificação,15


In [ ]:
demanda.head(53731)

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade
0,2022-01-01,1,Terminal Antônio Bezerra,4,Gratuidade Idoso,666
1,2022-01-01,1,Terminal Antônio Bezerra,5,Vale-transporte (vermelho),2
2,2022-01-01,1,Terminal Antônio Bezerra,11,Idoso Identificação,97
3,2022-01-01,1,Terminal Antônio Bezerra,12,Estudante com débito,873
4,2022-01-01,1,Terminal Antônio Bezerra,16,Grat. Deficiente - S/Acomp,4
...,...,...,...,...,...,...
53748,2022-01-20,349,Cj Veneza Tropical/Mirasol II,37,VT Beneficio Piloto BUM,1
53749,2022-01-20,350,Av. José Bastos/Lagoa,3,Estudante,13
53750,2022-01-20,350,Av. José Bastos/Lagoa,4,Gratuidade Idoso,447
53751,2022-01-20,350,Av. José Bastos/Lagoa,5,Vale-transporte (vermelho),3


Percebi aqui, que juntou-se os dataframes, mas o índice não é um só, ele volta pro 0 ao chegar na primeira linha do segundo dataframe

In [ ]:
demanda_2 = pd.merge(demanda_jan,dem_fev,how="outer")

In [ ]:
demanda_2.isna().sum()

Data               0
ID                 0
Nome               0
ID Passageiro      0
Tipo Passageiro    0
Quantidade         0
dtype: int64

In [ ]:
demanda_2

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade
0,2022-01-01,1,Terminal Antônio Bezerra,4,Gratuidade Idoso,666
1,2022-01-01,1,Terminal Antônio Bezerra,5,Vale-transporte (vermelho),2
2,2022-01-01,1,Terminal Antônio Bezerra,11,Idoso Identificação,97
3,2022-01-01,1,Terminal Antônio Bezerra,12,Estudante com débito,873
4,2022-01-01,1,Terminal Antônio Bezerra,16,Grat. Deficiente - S/Acomp,4
...,...,...,...,...,...,...
166457,2022-02-28,920,Papicu/Caça e Pesca,32,Gratuidade Infantil,1
166458,2022-02-28,920,Papicu/Caça e Pesca,37,VT Beneficio Piloto BUM,2
166459,2022-02-28,999,Quilometragem/Terminal,4,Gratuidade Idoso,2
166460,2022-02-28,999,Quilometragem/Terminal,11,Idoso Identificação,15


Aprendemos então que o merge une os dois dataframes incluindo o índice e o concat apenas cola os dois dataframes.

# 2) Quais as 5  linhas de ônibus de Fortaleza que mais passam passageiros nos dias da semana?

In [ ]:
qtd = demanda_2.groupby("ID").sum()["Quantidade"]
print(type(qtd))
ranking = demanda_2.groupby("ID").sum()["Quantidade"].rank(method="max")

<class 'pandas.core.series.Series'>


In [ ]:
qtd

ID
1      236709
2       25848
3      494176
4       77783
5       60809
        ...  
906    201328
907     51724
913     65482
920    124091
999      3564
Name: Quantidade, Length: 324, dtype: int64

### COMO FOI O PROCESSO?
Travei um pouco em como fazer, mas usei o groupby pra juntar tudo baseado no ID, com a função de soma, depois, baseado na coluna quantidade usei o comando rank para determinar quem tinha mais ou menos demanda.

In [ ]:
ranking.sort_values()

ID
20       1.0
40       2.0
774      3.0
95       4.0
326      5.0
       ...  
42     320.0
712    321.0
76     322.0
41     323.0
3      324.0
Name: Quantidade, Length: 324, dtype: float64

Depois rodei esse sort_values, me assustei com o resultado porque deu coincidentemente o Campus do Pici como o mais lotado. Mas, NINGUÉM PASSA NA CATRACA DO CAMPUS DO PICI. Percebi então que a coluna estava do menor para o maior, então devemos usar o argumento ascending

In [ ]:
ranking.sort_values(ascending=False).head(6)

ID
3      324.0
41     323.0
76     322.0
712    321.0
42     320.0
74     319.0
Name: Quantidade, dtype: float64

Agora sim, temos o ranking na ordem correta. Agora é só ver quais linhas se referem os index.

**DÚVIDA: COMO FAZER ISSO MAIS PYTHONICO?**

In [ ]:
demanda_2.loc[demanda_2["ID"]==3]["Nome"]

12        Terminal Parangaba I                    
13        Terminal Parangaba I                    
14        Terminal Parangaba I                    
15        Terminal Parangaba I                    
16        Terminal Parangaba I                    
                            ...                   
163605    Terminal Parangaba I                    
163606    Terminal Parangaba I                    
163607    Terminal Parangaba I                    
163608    Terminal Parangaba I                    
163609    Terminal Parangaba I                    
Name: Nome, Length: 766, dtype: object

In [ ]:
demanda_2.loc[demanda_2["ID"]==41]

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade
220,2022-01-01,41,Parangaba/Oliveira Paiva/Papicu,3,Estudante,5
221,2022-01-01,41,Parangaba/Oliveira Paiva/Papicu,4,Gratuidade Idoso,982
222,2022-01-01,41,Parangaba/Oliveira Paiva/Papicu,5,Vale-transporte (vermelho),21
223,2022-01-01,41,Parangaba/Oliveira Paiva/Papicu,9,Vt. Identificação,12
224,2022-01-01,41,Parangaba/Oliveira Paiva/Papicu,11,Idoso Identificação,116
...,...,...,...,...,...,...
163854,2022-02-28,41,Parangaba/Oliveira Paiva/Papicu,15,Projovem,11
163855,2022-02-28,41,Parangaba/Oliveira Paiva/Papicu,16,Grat. Deficiente - S/Acomp,175
163856,2022-02-28,41,Parangaba/Oliveira Paiva/Papicu,31,Vt. Institucional,5
163857,2022-02-28,41,Parangaba/Oliveira Paiva/Papicu,32,Gratuidade Infantil,2


In [ ]:
demanda_2.loc[demanda_2["ID"]==76]

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade
405,2022-01-01,76,Cj Ceará/Aldeota/Papicu,3,Estudante,5
406,2022-01-01,76,Cj Ceará/Aldeota/Papicu,4,Gratuidade Idoso,1021
407,2022-01-01,76,Cj Ceará/Aldeota/Papicu,5,Vale-transporte (vermelho),2
408,2022-01-01,76,Cj Ceará/Aldeota/Papicu,9,Vt. Identificação,14
409,2022-01-01,76,Cj Ceará/Aldeota/Papicu,11,Idoso Identificação,85
...,...,...,...,...,...,...
164116,2022-02-28,76,Cj Ceará/Aldeota/Papicu,15,Projovem,5
164117,2022-02-28,76,Cj Ceará/Aldeota/Papicu,16,Grat. Deficiente - S/Acomp,132
164118,2022-02-28,76,Cj Ceará/Aldeota/Papicu,31,Vt. Institucional,6
164119,2022-02-28,76,Cj Ceará/Aldeota/Papicu,32,Gratuidade Infantil,4


In [ ]:
demanda_2.loc[demanda_2["ID"]==712]

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade
1847,2022-01-01,712,Cj Palmeiras/Papicu,2,Popular,13
1848,2022-01-01,712,Cj Palmeiras/Papicu,3,Estudante,72
1849,2022-01-01,712,Cj Palmeiras/Papicu,4,Gratuidade Idoso,571
1850,2022-01-01,712,Cj Palmeiras/Papicu,5,Vale-transporte (vermelho),3
1851,2022-01-01,712,Cj Palmeiras/Papicu,11,Idoso Identificação,60
...,...,...,...,...,...,...
166137,2022-02-28,712,Cj Palmeiras/Papicu,15,Projovem,36
166138,2022-02-28,712,Cj Palmeiras/Papicu,16,Grat. Deficiente - S/Acomp,134
166139,2022-02-28,712,Cj Palmeiras/Papicu,31,Vt. Institucional,2
166140,2022-02-28,712,Cj Palmeiras/Papicu,32,Gratuidade Infantil,6


In [ ]:
demanda_2.loc[demanda_2["ID"]==42]

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade
231,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,3,Estudante,12
232,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,4,Gratuidade Idoso,964
233,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,5,Vale-transporte (vermelho),20
234,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,9,Vt. Identificação,23
235,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,11,Idoso Identificação,87
...,...,...,...,...,...,...
163866,2022-02-28,42,Antônio Bezerra/Francisco Sá/Papicu,15,Projovem,23
163867,2022-02-28,42,Antônio Bezerra/Francisco Sá/Papicu,16,Grat. Deficiente - S/Acomp,137
163868,2022-02-28,42,Antônio Bezerra/Francisco Sá/Papicu,31,Vt. Institucional,2
163869,2022-02-28,42,Antônio Bezerra/Francisco Sá/Papicu,32,Gratuidade Infantil,3


In [ ]:
demanda_2.loc[demanda_2["ID"]==74]

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade
387,2022-01-01,74,Antônio Bezerra/Unifor,3,Estudante,3
388,2022-01-01,74,Antônio Bezerra/Unifor,4,Gratuidade Idoso,434
389,2022-01-01,74,Antônio Bezerra/Unifor,5,Vale-transporte (vermelho),14
390,2022-01-01,74,Antônio Bezerra/Unifor,9,Vt. Identificação,3
391,2022-01-01,74,Antônio Bezerra/Unifor,11,Idoso Identificação,57
...,...,...,...,...,...,...
164095,2022-02-28,74,Antônio Bezerra/Unifor,16,Grat. Deficiente - S/Acomp,123
164096,2022-02-28,74,Antônio Bezerra/Unifor,31,Vt. Institucional,3
164097,2022-02-28,74,Antônio Bezerra/Unifor,32,Gratuidade Infantil,3
164098,2022-02-28,74,Antônio Bezerra/Unifor,33,Vale-Trasnporte ENEM,1


In [ ]:
linhas = {"041 - Parangaba/Oliveira Paiva/Papicu":qtd[41],"076 - Conjunto Ceará/Aldeota/Papicu":qtd[76],"712 - Conjunto Palmeiras/Papicu":qtd[712],"042 - Antônio Bezerra/Francisco Sá/Papicu":qtd[42],"074 - Antônio Bezerra/Unifor": qtd[74]}

linhas

{'041 - Parangaba/Oliveira Paiva/Papicu': 452383,
 '042 - Antônio Bezerra/Francisco Sá/Papicu': 394844,
 '074 - Antônio Bezerra/Unifor': 356597,
 '076 - Conjunto Ceará/Aldeota/Papicu': 431214,
 '712 - Conjunto Palmeiras/Papicu': 429077}

# 3) Quais as 5 linhas que possuem a maior demanda de passageiros nos finais de semana?

In [ ]:
demanda_2.groupby("ID").sum()["Quantidade"].shape

(324,)

Um desafio é saber quais dias são da semana e quais dias são fins de semana. Vamos tentar usando o bdate_range, pra isso vamos ver se a coluna data é do tipo data.

In [ ]:
demanda_2.dtypes
demanda_2["Data"] = pd.to_datetime(demanda_2["Data"])


In [ ]:
demanda_2.dtypes

Data               datetime64[ns]
ID                          int64
Nome                       object
ID Passageiro               int64
Tipo Passageiro            object
Quantidade                  int64
dtype: object

In [ ]:
def findWeekend(df):
  res = len(pd.bdate_range(df["Data"],df["Data"]))
  if res==0:
    return True
  else:
    return False


O pd.bdata_range retorna uma "lista" de datas que não são finais de semana. Portanto, eu peguei o tamanho dessa lista, se fosse 0 é porque a data é um fim de semana, se fosse 1 é porque a data era um dia da semana. Fiz essa lógica dentro de uma função que usava o dataframe como parâmetro.

In [ ]:
demanda_2["Final de semana"] = demanda_2.apply(findWeekend, axis=1)

Depois, apliquei a uma nova coluna chamada Final de Semana a função apply que roda a função que está como parâmetro e passa o valor para a nova coluna.

In [ ]:
df_mask = demanda_2["Final de semana"] == True
demanda_fds = demanda_2[df_mask]

In [ ]:
demanda_fds.shape

(43043, 7)

In [ ]:

ranking_fds = demanda_fds.groupby("ID").sum()["Quantidade"].rank(method="max")

In [ ]:

demanda_fds.groupby("ID").sum()["Quantidade"].shape

(294,)

30 linhas de ônibus não apareceram no dataframe, possivelmente porque não funcionam em um fim de semana.

In [ ]:
ranking_fds.sort_values(ascending=False).head(6)

ID
52     294.0
51     293.0
41     292.0
3      291.0
712    290.0
42     289.0
Name: Quantidade, dtype: float64

In [ ]:
demanda_fds.loc[demanda_fds["ID"]==42]

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade,Final de semana
231,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,3,Estudante,12,True
232,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,4,Gratuidade Idoso,964,True
233,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,5,Vale-transporte (vermelho),20,True
234,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,9,Vt. Identificação,23,True
235,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,11,Idoso Identificação,87,True
...,...,...,...,...,...,...,...
161555,2022-02-27,42,Antônio Bezerra/Francisco Sá/Papicu,16,Grat. Deficiente - S/Acomp,83,True
161556,2022-02-27,42,Antônio Bezerra/Francisco Sá/Papicu,27,Pagante Horário Diferenciado,274,True
161557,2022-02-27,42,Antônio Bezerra/Francisco Sá/Papicu,31,Vt. Institucional,7,True
161558,2022-02-27,42,Antônio Bezerra/Francisco Sá/Papicu,32,Gratuidade Infantil,3,True


### Quais as 5 linhas que possuem a maior demanda de passageiros totais nos dias da semana?

In [ ]:
df_mask = demanda_2["Final de semana"] == False
demanda_bday = demanda_2[df_mask]

In [ ]:
demanda_bday.shape

(123419, 7)

In [ ]:

ranking_bday = demanda_bday.groupby("ID").sum()["Quantidade"].rank(method="max")

In [ ]:
ranking_bday.shape

(317,)

In [ ]:
ranking_bday.sort_values(ascending=False).head(6)

ID
3      317.0
76     316.0
41     315.0
712    314.0
42     313.0
74     312.0
Name: Quantidade, dtype: float64

In [ ]:
demanda_fds.loc[demanda_fds["ID"]==42]

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade,Final de semana
231,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,3,Estudante,12,True
232,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,4,Gratuidade Idoso,964,True
233,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,5,Vale-transporte (vermelho),20,True
234,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,9,Vt. Identificação,23,True
235,2022-01-01,42,Antônio Bezerra/Francisco Sá/Papicu,11,Idoso Identificação,87,True
...,...,...,...,...,...,...,...
161555,2022-02-27,42,Antônio Bezerra/Francisco Sá/Papicu,16,Grat. Deficiente - S/Acomp,83,True
161556,2022-02-27,42,Antônio Bezerra/Francisco Sá/Papicu,27,Pagante Horário Diferenciado,274,True
161557,2022-02-27,42,Antônio Bezerra/Francisco Sá/Papicu,31,Vt. Institucional,7,True
161558,2022-02-27,42,Antônio Bezerra/Francisco Sá/Papicu,32,Gratuidade Infantil,3,True


# 4) Qual a proporção de pagantes x gratuidades em Fortaleza.

Primeiro vamos ver quantos ID de passageiros existem, e avaliar quais são pagos e gratuitos.

In [ ]:
demanda_2

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade,Final de semana
0,2022-01-01,1,Terminal Antônio Bezerra,4,Gratuidade Idoso,666,True
1,2022-01-01,1,Terminal Antônio Bezerra,5,Vale-transporte (vermelho),2,True
2,2022-01-01,1,Terminal Antônio Bezerra,11,Idoso Identificação,97,True
3,2022-01-01,1,Terminal Antônio Bezerra,12,Estudante com débito,873,True
4,2022-01-01,1,Terminal Antônio Bezerra,16,Grat. Deficiente - S/Acomp,4,True
...,...,...,...,...,...,...,...
166457,2022-02-28,920,Papicu/Caça e Pesca,32,Gratuidade Infantil,1,False
166458,2022-02-28,920,Papicu/Caça e Pesca,37,VT Beneficio Piloto BUM,2,False
166459,2022-02-28,999,Quilometragem/Terminal,4,Gratuidade Idoso,2,False
166460,2022-02-28,999,Quilometragem/Terminal,11,Idoso Identificação,15,False


In [ ]:
tipos_passageiro = demanda_2["ID Passageiro"].unique()

In [ ]:
tipos_passageiro

array([ 4,  5, 11, 12, 16, 37,  2, 27,  3,  6,  9, 15, 31, 32, 28, 33,  7])

In [ ]:
len(tipos_passageiro)










17

Temos 17 tipos diferentes de passageiros (maneiras de pagar passagem.

O código abaixo separa as linhas pelo tipo de passageiro e soma as quantidades gerando o seguinte dataframe:

In [ ]:
df_tipo_passageiro = demanda_2.groupby("Tipo Passageiro").sum()["Quantidade"]

In [ ]:
df_tipo_passageiro

Tipo Passageiro
Crachá Empresas                        7583
Estudante                            330529
Estudante com débito                8281646
Grat. Deficiente - S/Acomp           718802
Gratuidade Identificação Ettusa           4
Gratuidade Idoso                   12740376
Gratuidade Infantil                   22060
Idoso Identificação                 3045964
Pagante Horário Diferenciado         202663
Popular                               95857
Projovem                              72209
Transbordo                            18315
VT Beneficio Piloto BUM              107807
Vale-Trasnporte ENEM                    177
Vale-transporte (vermelho)            94711
Vt. Identificação                    279306
Vt. Institucional                     24417
Name: Quantidade, dtype: int64

Depois somamos as quantidades de passageiros com gratuidade. Para a meia passagem dividimos a quantidade por dois para ficar metade das pessoas com valor cheio e metade gratuito.

In [ ]:
pagos = df_tipo_passageiro.iloc[[8,9,10,11,12,13,14,15,16]].sum() + \
              df_tipo_passageiro[["Estudante","Estudante com débito"]].sum()*0.5

In [ ]:
gratuidades = df_tipo_passageiro.iloc[[0,3,4,5,6,7]].sum() + \
              df_tipo_passageiro[["Estudante","Estudante com débito"]].sum()*0.5

In [ ]:
gratuidades

20840876.5

In [ ]:
total = df_tipo_passageiro.sum() ###total de quantidades juntando tudo

In [ ]:
pct_gratis = gratuidades/total
print(pct_gratis*100)

80.02663231144442


In [ ]:
pct_pago = pagos/total
print(pct_pago*100)

19.97336768855559


In [ ]:
pct_pago + pct_gratis ##conferindo se o valor bate.

1.0

# Ranking por número porcentagem de estudantes e idosos

## 1 - Estudantes

In [ ]:
demanda_2.head(20)

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade,Final de semana
0,2022-01-01,1,Terminal Antônio Bezerra,4,Gratuidade Idoso,666,True
1,2022-01-01,1,Terminal Antônio Bezerra,5,Vale-transporte (vermelho),2,True
2,2022-01-01,1,Terminal Antônio Bezerra,11,Idoso Identificação,97,True
3,2022-01-01,1,Terminal Antônio Bezerra,12,Estudante com débito,873,True
4,2022-01-01,1,Terminal Antônio Bezerra,16,Grat. Deficiente - S/Acomp,4,True
5,2022-01-01,1,Terminal Antônio Bezerra,37,VT Beneficio Piloto BUM,23,True
6,2022-01-01,2,Catraca Móvel/Terminal/Antônio Bezerra,2,Popular,44,True
7,2022-01-01,2,Catraca Móvel/Terminal/Antônio Bezerra,4,Gratuidade Idoso,16,True
8,2022-01-01,2,Catraca Móvel/Terminal/Antônio Bezerra,11,Idoso Identificação,23,True
9,2022-01-01,2,Catraca Móvel/Terminal/Antônio Bezerra,12,Estudante com débito,5,True


In [ ]:
df_estudante = demanda_2[(demanda_2["ID Passageiro"] == 12) | (demanda_2["ID Passageiro"]==3)]

In [ ]:
df_nestudante = df_estudante.groupby("ID").sum()["Quantidade"]

In [ ]:
df_nestudante.columns = ["ID", "Estudantes"]

In [ ]:
df_nestudante = df_nestudante.reset_index()

In [ ]:
df_nestudante

,ID,Quantidade
0,1,69239
1,2,2942
2,3,134822
3,4,31360
4,5,22979
...,...,...
319,906,72432
320,907,19206
321,913,21169
322,920,42422


In [ ]:
gb_total = demanda_2.groupby("ID").sum()["Quantidade"]

In [ ]:
gb_total = gb_total.reset_index()

In [ ]:
df_nestudante

,ID,Quantidade
0,1,69239
1,2,2942
2,3,134822
3,4,31360
4,5,22979
...,...,...
319,906,72432
320,907,19206
321,913,21169
322,920,42422


In [ ]:
df_nestudante["Total"] = gb_total["Quantidade"]

In [ ]:
gb_total

,ID,Quantidade
0,1,236709
1,2,25848
2,3,494176
3,4,77783
4,5,60809
...,...,...
319,906,201328
320,907,51724
321,913,65482
322,920,124091


In [ ]:
df_nestudante

,ID,Quantidade,Total
0,1,69239,236709
1,2,2942,25848
2,3,134822,494176
3,4,31360,77783
4,5,22979,60809
...,...,...,...
319,906,72432,201328
320,907,19206,51724
321,913,21169,65482
322,920,42422,124091


In [ ]:
rank_estudante = df_estudante.groupby("ID").sum()["Quantidade"].rank(method="max")

In [ ]:
df_nestudante

,ID,Quantidade,Total
0,1,69239,236709
1,2,2942,25848
2,3,134822,494176
3,4,31360,77783
4,5,22979,60809
...,...,...,...
319,906,72432,201328
320,907,19206,51724
321,913,21169,65482
322,920,42422,124091


In [ ]:
def percentual(df):
  return (df["Quantidade"]/df["Total"])*100

df_nestudante["Percentual"] = df_nestudante.apply(percentual, axis=1)

In [ ]:
rank_estudante = df_nestudante[["ID","Percentual"]].rank(method="min")
rank_estudante.sort_values(by="Percentual",ascending=False).head(11)

,ID,Percentual
75,76.0,324.0
261,262.0,323.0
306,307.0,322.0
317,318.0,321.0
275,276.0,320.0
245,246.0,319.0
208,209.0,318.0
298,299.0,317.0
301,302.0,316.0
305,306.0,315.0


In [ ]:
df_nestudante.loc[df_nestudante["ID"]==310]

,ID,Quantidade,Total,Percentual
120,310,8015,26533,30.207666


In [ ]:
rank_estudante.sort_values(ascending=False).head(11)

ID
41     324.0
712    323.0
76     322.0
3      321.0
42     320.0
51     319.0
52     318.0
50     317.0
74     316.0
755    315.0
45     314.0
Name: Quantidade, dtype: float64

## 2 - Idosos

In [ ]:
df_idosos = demanda_2[demanda_2["ID Passageiro"] == 4]

In [ ]:
df_idosos

,Data,ID,Nome,ID Passageiro,Tipo Passageiro,Quantidade,Final de semana
0,2022-01-01,1,Terminal Antônio Bezerra,4,Gratuidade Idoso,666,True
7,2022-01-01,2,Catraca Móvel/Terminal/Antônio Bezerra,4,Gratuidade Idoso,16,True
14,2022-01-01,3,Terminal Parangaba I,4,Gratuidade Idoso,696,True
26,2022-01-01,5,Terminal Lagoa,4,Gratuidade Idoso,161,True
32,2022-01-01,6,Terminal Messejana,4,Gratuidade Idoso,501,True
...,...,...,...,...,...,...,...
166420,2022-02-28,906,Caça e Pesca/Serviluz/Centro,4,Gratuidade Idoso,1611,False
166431,2022-02-28,907,Castelo Encantado/Centro,4,Gratuidade Idoso,304,False
166439,2022-02-28,913,Papicu/Serviluz/Varjota,4,Gratuidade Idoso,574,False
166450,2022-02-28,920,Papicu/Caça e Pesca,4,Gratuidade Idoso,1156,False


In [ ]:
rank_idosos = df_idosos.groupby("ID").sum()["Quantidade"].rank(method="max")

In [ ]:
rank_idosos.sort_values(ascending=False).head(11)

ID
3      323.0
76     322.0
41     321.0
42     320.0
712    319.0
51     318.0
52     317.0
74     316.0
755    315.0
45     314.0
855    313.0
Name: Quantidade, dtype: float64

## 2 - Idosos